# Tarea: Ataque con Rainbow Tables a MD5

**Asignatura:** Criptografía (3.º Ingeniería Matemática)  
**Objetivo general:** Investigar y experimentar con ataques a la integridad de contraseñas basados en rainbow tables aplicados al algoritmo MD5.

---

## Enunciado

### Contexto  
Las rainbow tables son una técnica de pre‑cómputo que permite invertir funciones hash (como MD5) a costa de memoria adicional. En esta práctica, limitaremos el espacio de contraseñas a todas las cadenas de 4 dígitos (`"0000"`–`"9999"`) para atacar un hash objetivo y detectar colisiones.

### Pasos de la práctica

1. **Construcción de la Rainbow Table**  
   - Para cada contraseña de 4 dígitos, generar una **cadena de transformación** de longitud 4:  
     1. Calcular MD5 del valor actual.  
     2. Reducir (usando los primeros 8 dígitos del hash + número de ronda) a un número 0000–9999.  
   - Almacenar en un diccionario `{ endpoint → contraseña_inicial }`.  
   - Documentar en el informe cuántas entradas hay y si surgieron colisiones en el endpoint durante el pre‑cómputo.

2. **Obtención del Hash Objetivo y Generación de la Cadena del Target**  
   - Elegir una contraseña real.  
   - Calcular su cadena de transformación y anotar el **endpoint**.

3. **Búsqueda en la Rainbow Table**  
   - Buscar ese endpoint en la tabla pre‑computada.  
   - **Detectar colisiones y falso positivo**.

4. **Verificación y Análisis de Colisiones**  

### Entregables - El trabajo se realizará sobre esta misma plantilla de forma individual

1. **Informe Markdown y código Python** que incluya:    
   - Código Python completo y comentado.  
   - Discusión de resultados y conclusiones.
   - Tener ejecutada todas las celdas para facilitar su correción


---

## Rúbrica de Evaluación

| Descriptor                    | Insuficiente (1–3)                                                                                                   | Satisfactorio (4–7)                                                                                                                                       | Sobresaliente (8–10)                                                                                                                                            |
|-------------------------------|-----------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **1. Comprensión teórica**    | Conceptos de rainbow tables y MD5 confusos o erróneos; no identifica correctamente colisiones ni trade‑off tiempo/memoria. | Define correctamente rainbow tables y MD5; explica los pasos del ataque; menciona colisiones, aunque con vacíos conceptuales.                               | Explica con claridad y profundidad los conceptos: Merkle–Damgård, función de reducción, colisiones, trade‑off; aporta referencias y ejemplos adicionales.        |
| **2. Implementación práctica**| Código incompleto, sin comentarios o con errores que impiden la ejecución; no muestra pasos en pantalla.               | Código funcional que genera la tabla y ataca el hash; incluye comentarios básicos y muestra salidas por pantalla.                                          | Código muy bien estructurado y documentado; usa funciones reutilizables; muestra interactivamente cada paso con mensajes claros y capturas o gráficos si procede. |
| **3. Análisis de resultados** | Informe sin discusión de resultados, o conclusiones equivocadas; no detecta ni explica colisiones.                   | Presenta resultados de forma clara; detecta colisiones y ofrece una breve reflexión sobre sus causas.                                                    | Análisis crítico y profundo: cuantifica colisiones, compara tasas de éxito, discute cómo ampliaría el espacio o usaría salt para mitigar ataques; propone mejoras. |

---
**Fecha de entrega:** 13 de mayo  
**Formato:** Markdown + código  (Notebook Jupyter)
**Peso en la nota final:** 20% de la evaluación de la asignatura en su parte ordinaria (40%) 


## 🔧 Funciones de Hashing y Reducción

A continuación se definen tres funciones clave para construir la Rainbow Table:

### 1. `md5_hash(texto)`
Esta función aplica el algoritmo MD5 a una cadena de texto y devuelve su hash en formato hexadecimal.

- **Entrada**: una contraseña como `"1234"`.
- **Salida**: un hash MD5 como `"81dc9bdb52d04dc20036dbd8313ed055"`.

### 2. `reducir(hash_hex, ronda)`
Transforma un hash largo (hexadecimal) en una contraseña de 4 dígitos (`"0000"` a `"9999"`), usando una función de reducción.

- Toma los primeros 8 caracteres del hash.
- Los convierte a un número entero en base 16.
- Le suma el número de ronda para evitar repeticiones.
- Lo reduce al rango 0000–9999 usando módulo.
- Devuelve ese número como string de 4 cifras con ceros a la izquierda.

Este paso es fundamental en las Rainbow Tables para pasar de un hash largo a una clave corta dentro del espacio de búsqueda.

### 3. `calcular_endpoint(contraseña_inicial)`
Aplica 4 veces el proceso de hash y reducción para una contraseña dada. Devuelve el **endpoint**, que es el resultado final después de esas 4 transformaciones.

Este endpoint será la clave usada en la Rainbow Table para asociarla con su contraseña inicial.

---


In [1]:
import hashlib

# Función que aplica MD5 a una cadena y devuelve el hash en hexadecimal
def md5_hash(texto):
    return hashlib.md5(texto.encode()).hexdigest()

# Función de reducción: convierte un hash en una contraseña de 4 dígitos
def reducir(hash_hex, ronda):
    # Cogemos los primeros 8 caracteres del hash (32 bits)
    sub_hash = hash_hex[:8]
    # Lo convertimos a número entero y le sumamos la ronda para evitar colisiones repetidas
    num = int(sub_hash, 16) + ronda
    # Lo llevamos al rango 0000-9999 con módulo
    reducido = str(num % 10000).zfill(4)
    return reducido

# Función que calcula el endpoint de una contraseña haciendo 4 pasos de hash+reducción
def calcular_endpoint(contraseña_inicial):
    actual = contraseña_inicial
    for ronda in range(4):
        hash_actual = md5_hash(actual)
        actual = reducir(hash_actual, ronda)
    return actual  # Este es el endpoint final de la cadena

# Creamos la tabla rainbow
rainbow_table = {}

# Recorremos todas las contraseñas de 0000 a 9999
for i in range(10000):
    inicio = str(i).zfill(4)
    endpoint = calcular_endpoint(inicio)
    
    # Guardamos solo si el endpoint aún no está (evita colisiones)
    if endpoint not in rainbow_table:
        rainbow_table[endpoint] = inicio

# Mostrar algunos ejemplos de la tabla
for i, (end, start) in enumerate(rainbow_table.items()):
    print(f"{i+1}: {start} -> {end}")
    if i == 9:  # solo muestra los primeros 10
        break

# Información general
print(f"\nTamaño total de la Rainbow Table: {len(rainbow_table)} entradas")


1: 0000 -> 5961
2: 0001 -> 1512
3: 0002 -> 1660
4: 0003 -> 9301
5: 0004 -> 7161
6: 0005 -> 9755
7: 0006 -> 8244
8: 0008 -> 7529
9: 0009 -> 6624
10: 0010 -> 0072

Tamaño total de la Rainbow Table: 3157 entradas


## 📉 Colisiones durante la construcción de la Rainbow Table

Durante la generación de la Rainbow Table se recorrieron todas las contraseñas posibles de 4 dígitos, es decir, un total de **10.000 combinaciones** (`"0000"` a `"9999"`). Para cada una de ellas, se aplicó una cadena de 4 pasos de hash y reducción, y se almacenó el `endpoint` final asociado a la contraseña inicial.

Sin embargo, al finalizar la construcción, observamos que la tabla contiene **solo 3.157 entradas únicas**. Esto significa que se han producido **colisiones en los endpoints**, es decir, varias contraseñas distintas han terminado generando el mismo `endpoint`.

Dado que en la implementación de la tabla solo se guarda el primer valor asociado a cada endpoint (usando un diccionario), las colisiones provocan que se descarten las contraseñas repetidas. En consecuencia, **se han perdido aproximadamente 6.843 combinaciones posibles**, lo que refleja el clásico **compromiso entre memoria y cobertura** que presentan las Rainbow Tables.

Este resultado demuestra que, aunque la Rainbow Table es más eficiente en memoria que una tabla exhaustiva, **no garantiza cobertura total del espacio de claves**.


In [3]:
# Repetimos el proceso de obtener el endpoint real
def obtener_endpoint_real(contraseña_real):
    actual = contraseña_real
    for ronda in range(4):
        hash_actual = md5_hash(actual)
        actual = reducir(hash_actual, ronda)
    return actual

# Vamos a probar con una lista de contraseñas y ver si hay colisiones en sus endpoints

# Lista de 5 contraseñas a probar
contrasenas_a_probar = ["1234", "5678", "0000", "2025", "9999"]

# Diccionario para guardar resultados
endpoints_info = []

for pwd in contrasenas_a_probar:
    endpoint = obtener_endpoint_real(pwd)
    inicio_posible = rainbow_table.get(endpoint, None)
    colision = (inicio_posible != pwd) if inicio_posible else None
    endpoints_info.append({
        "Contraseña_real": pwd,
        "Endpoint_obtenido": endpoint,
        "En_tabla": endpoint in rainbow_table,
        "Inicio_guardado_en_tabla": inicio_posible,
        "¿Hay_colisión?": colision
    })

# Mostrar resultados
import pandas as pd
df_endpoints = pd.DataFrame(endpoints_info)
df_endpoints



,Contraseña_real,Endpoint_obtenido,En_tabla,Inicio_guardado_en_tabla,¿Hay_colisión?
0,1234,5595,True,1234,False
1,5678,3002,True,0665,True
2,0000,5961,True,0000,False
3,2025,8232,True,0214,True
4,9999,8113,True,1327,True


## 🔍 Detección de colisiones en endpoints

Para analizar cómo afectan las colisiones a la Rainbow Table, se seleccionaron 5 contraseñas distintas: `"1234"`, `"5678"`, `"0000"`, `"2025"` y `"9999"`. Para cada una de ellas, se aplicaron 4 pasos de hash y reducción para obtener su `endpoint`. Luego, se comprobó si dicho endpoint ya estaba registrado en la Rainbow Table y, en caso afirmativo, con qué contraseña inicial estaba asociado.

### 📋 Resultados

| Contraseña real | Endpoint obtenido | ¿Está en la tabla? | Contraseña guardada | ¿Hay colisión? |
|------------------|-------------------|----------------------|----------------------|----------------|
| 1234             | 5595               | Sí                   | 1234                 | No             |
| 5678             | 3002               | Sí                   | distinta             | Sí             |
| 0000             | 5961               | Sí                   | 0000                 | No             |
| 2025             | 8232               | Sí/No                | distinta                  | Sí          |
| 9999             | 8113               | Sí                   | distinta             | Sí             |

*(Nota: los endpoints exactos y los valores reales de colisión se generan automáticamente con el código Python.)*

### 🧠 Conclusión

- Algunas contraseñas como `"1234"` y `"0000"` no presentan colisión: el endpoint que generan está en la tabla y fue generado por ellas mismas.
- Otras como `"5678"`, `"2025"`  y `"9999"` sí presentan colisión: su endpoint ya estaba en la tabla pero asociado a otra contraseña distinta.
- Este comportamiento refleja una limitación clave de las Rainbow Tables: si varias contraseñas generan el mismo endpoint, **solo se conserva la primera**, y el resto se pierde, haciendo imposible su recuperación.

Esto evidencia el compromiso entre eficiencia y cobertura que implica este tipo de ataques por precomputación.
